## This is the training the model after i have defined _make_spec() method in DJSSP and after i have defined machine_breakdowns shape there.

In [6]:
import time

from rl4co.envs.scheduling.djssp.env import DJSSPEnv
from rl4co.models import L2DPolicy, L2DModel
from rl4co.utils import RL4COTrainer
import gc
from rl4co.envs import JSSPEnv
from rl4co.models.zoo.l2d.model import L2DPPOModel
from rl4co.models.zoo.l2d.policy import L2DPolicy4PPO
from torch.utils.data import DataLoader
import json
import os
%load_ext autoreload
%autoreload 2
generator_params = {
"num_jobs" : 8 ,
"num_machines": 8 ,
"min_processing_time": 1 ,
"max_processing_time": 99 ,
"mtbf" : 17 ,
"mttr" : 4
}
env = DJSSPEnv(generator_params=generator_params,
_torchrl_mode=True,
stepwise_reward=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import torch
if torch.cuda.is_available():
    accelerator = "gpu"
    batch_size = 4
    train_data_size = 2_000
    embed_dim = 128
    num_encoder_layers = 4
else:
    accelerator = "cpu"
    batch_size = 4
    train_data_size = 1_000
    embed_dim = 64
    num_encoder_layers = 2

In [8]:
# Policy: neural network, in this case with encoder-decoder architecture
policy = L2DPolicy4PPO(
    embed_dim=embed_dim,
    num_encoder_layers=num_encoder_layers,
    env_name="djssp",
    het_emb=False
)

In [9]:
model = L2DPPOModel(
    env=env,
    policy=policy,
    batch_size=batch_size,
    train_data_size=train_data_size,
    val_data_size=1,
    optimizer_kwargs={"lr": 1e-4}
)

In [10]:
# CHECKPOINT_PATH = "last.ckpt"
device = "cuda" if torch.cuda.is_available() else "cpu"

trainer = RL4COTrainer(
    max_epochs=1,
    accelerator=accelerator,
    devices=1,
    logger=None,
)

trainer.fit(model)

model = model.to(device)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Overriding gradient_clip_val to None for 'automatic_optimization=False' models
val_file not set. Generating dataset instead
test_file not set. Generating dataset instead


Epoch 0:   0%|          | 0/334 [08:02<?, ?it/s]



  | Name       | Type          | Params | Mode 
-----------------------------------------------------
0 | env        | DJSSPEnv      | 0      | train
1 | policy     | L2DPolicy4PPO | 25.5 K | train
2 | policy_old | L2DPolicy4PPO | 25.5 K | train
-----------------------------------------------------
51.1 K    Trainable params
0         Non-trainable params
51.1 K    Total params
0.204     Total estimated model params size (MB)
69        Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 250/250 [14:23<00:00,  0.29it/s, v_num=17, train/loss=208.0, train/surrogate_loss=-0.567, train/value_loss=417.0, train/entropy=0.230] 
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 250/250 [14:24<00:00,  0.29it/s, v_num=17, train/loss=208.0, train/surrogate_loss=-0.567, train/value_loss=417.0, train/entropy=0.230, val/reward=-926.]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 250/250 [14:24<00:00,  0.29it/s, v_num=17, train/loss=208.0, train/surrogate_loss=-0.567, train/value_loss=417.0, train/entropy=0.230, val/reward=-926.]
